In [28]:
"""
A minimal implementation of Monte Carlo tree search (MCTS) in Python 3
Luke Harold Miles, July 2019, Public Domain Dedication
See also https://en.wikipedia.org/wiki/Monte_Carlo_tree_search
https://gist.github.com/qpwo/c538c6f73727e254fdc7fab81024f6e1
"""
from abc import ABC, abstractmethod
from collections import defaultdict
import math



def normalize_notation(card: str):
    card = card.lower()
    if card[-1] not in ['s','c','h','d']:
        return "Suit Type Invalid"
    if card[:-1] not in ["2","3","4","5","6","7","8","9","t","j","q","k","a"]:
        return "Card Number Invalid"
    return card






class MCTS:
    "Monte Carlo tree searcher. First rollout the tree then choose a move."

    def __init__(self, exploration_weight=1):
        self.Q = defaultdict(int)  # total reward of each node
        self.N = defaultdict(int)  # total visit count for each node
        self.children = dict()  # children of each node
        self.exploration_weight = exploration_weight

    def choose(self, node):
        "Choose the best successor of node. (Choose a move in the game)"
        if node.is_terminal():
            raise RuntimeError(f"choose called on terminal node {node}")

        if node not in self.children:
            return node.find_random_child()

        def score(n):
            if self.N[n] == 0:
                return float("-inf")  # avoid unseen moves
            return self.Q[n] / self.N[n]  # average reward

        return max(self.children[node], key=score)

    def do_rollout(self, node):
        "Make the tree one layer better. (Train for one iteration.)"
        path = self._select(node)
        leaf = path[-1]
        self._expand(leaf)
        reward = self._simulate(leaf)
        self._backpropagate(path, reward)

    def _select(self, node):
        "Find an unexplored descendent of `node`"
        path = []
        while True:
            path.append(node)
            if node not in self.children or not self.children[node]:
                # node is either unexplored or terminal
                return path
            unexplored = self.children[node] - self.children.keys()
            if unexplored:
                n = unexplored.pop()
                path.append(n)
                return path
            node = self._uct_select(node)  # descend a layer deeper

    def _expand(self, node):
        "Update the `children` dict with the children of `node`"
        if node in self.children:
            return  # already expanded
        self.children[node] = node.find_children()

    def _simulate(self, node):
        "Returns the reward for a random simulation (to completion) of `node`"
        invert_reward = True
        while True:
            if node.is_terminal():
                reward = node.reward()
                return 1 - reward if invert_reward else reward
            node = node.find_random_child()
            invert_reward = not invert_reward

    def _backpropagate(self, path, reward):
        "Send the reward back up to the ancestors of the leaf"
        for node in reversed(path):
            self.N[node] += 1
            self.Q[node] += reward
            reward = 1 - reward  # 1 for me is 0 for my enemy, and vice versa

    def _uct_select(self, node):
        "Select a child of node, balancing exploration & exploitation"

        # All children of node should already be expanded:
        assert all(n in self.children for n in self.children[node])

        log_N_vertex = math.log(self.N[node])

        def uct(n):
            "Upper confidence bound for trees"
            return self.Q[n] / self.N[n] + self.exploration_weight * math.sqrt(
                log_N_vertex / self.N[n]
            )

        return max(self.children[node], key=uct)



# Abstract MCTS and Node

In [29]:
"""
A minimal implementation of Monte Carlo tree search (MCTS) in Python 3
Luke Harold Miles, July 2019, Public Domain Dedication
See also https://en.wikipedia.org/wiki/Monte_Carlo_tree_search
https://gist.github.com/qpwo/c538c6f73727e254fdc7fab81024f6e1
"""
from abc import ABC, abstractmethod
from collections import defaultdict
import math


class MCTS:
    "Monte Carlo tree searcher. First rollout the tree then choose a move."

    def __init__(self, exploration_weight=1):
        self.Q = defaultdict(int)  # total reward of each node
        self.N = defaultdict(int)  # total visit count for each node
        self.children = dict()  # children of each node
        self.exploration_weight = exploration_weight

    def choose(self, node):
        "Choose the best successor of node. (Choose a move in the game)"
        if node.is_terminal():
            raise RuntimeError(f"choose called on terminal node {node}")

        if node not in self.children:
            return node.find_random_child()

        def score(n):
            if self.N[n] == 0:
                return float("-inf")  # avoid unseen moves
            return self.Q[n] / self.N[n]  # average reward

        return max(self.children[node], key=score)

    def do_rollout(self, node):
        "Make the tree one layer better. (Train for one iteration.)"
        path = self._select(node)
        leaf = path[-1]
        self._expand(leaf)
        reward = self._simulate(leaf)
        self._backpropagate(path, reward)

    def _select(self, node):
        "Find an unexplored descendent of `node`"
        path = []
        while True:
            path.append(node)
            if node not in self.children or not self.children[node]:
                # node is either unexplored or terminal
                return path
            unexplored = self.children[node] - self.children.keys()
            if unexplored:
                n = unexplored.pop()
                path.append(n)
                return path
            node = self._uct_select(node)  # descend a layer deeper

    def _expand(self, node):
        "Update the `children` dict with the children of `node`"
        if node in self.children:
            return  # already expanded
        self.children[node] = node.find_children()

    def _simulate(self, node):
        "Returns the reward for a random simulation (to completion) of `node`"
        invert_reward = True
        while True:
            if node.is_terminal():
                reward = node.reward()
                return 1 - reward if invert_reward else reward
            node = node.find_random_child()
            invert_reward = not invert_reward

    def _backpropagate(self, path, reward):
        "Send the reward back up to the ancestors of the leaf"
        for node in reversed(path):
            self.N[node] += 1
            self.Q[node] += reward
            reward = 1 - reward  # 1 for me is 0 for my enemy, and vice versa

    def _uct_select(self, node):
        "Select a child of node, balancing exploration & exploitation"

        # All children of node should already be expanded:
        assert all(n in self.children for n in self.children[node])

        log_N_vertex = math.log(self.N[node])

        def uct(n):
            "Upper confidence bound for trees"
            return self.Q[n] / self.N[n] + self.exploration_weight * math.sqrt(
                log_N_vertex / self.N[n]
            )

        return max(self.children[node], key=uct)


class Node(ABC):
    """
    A representation of a single board state.
    MCTS works by constructing a tree of these Nodes.
    Could be e.g. a chess or checkers board state.
    """

    @abstractmethod
    def find_children(self):
        "All possible successors of this board state"
        return set()

    @abstractmethod
    def find_random_child(self):
        "Random successor of this board state (for more efficient simulation)"
        return None

    @abstractmethod
    def is_terminal(self):
        "Returns True if the node has no children"
        return True

    @abstractmethod
    def reward(self):
        "Assumes `self` is terminal node. 1=win, 0=loss, .5=tie, etc"
        return 0

    @abstractmethod
    def __hash__(self):
        "Nodes must be hashable"
        return 123456789

    @abstractmethod
    def __eq__(node1, node2):
        "Nodes must be comparable"
        return True

In [30]:
import random

In [31]:
"""
An example implementation of the abstract Node class for use in MCTS
If you run this file then you can play against the computer.
A tic-tac-toe board is represented as a tuple of 9 values, each either None,
True, or False, respectively meaning 'empty', 'X', and 'O'.
The board is indexed by row:
0 1 2
3 4 5
6 7 8
For example, this game board
O - X
O X -
X - -
corrresponds to this tuple:
(False, None, True, False, True, None, True, None, None)
"""

from collections import namedtuple
from random import choice

_TTTB = namedtuple("TicTacToeBoard", "tup turn winner terminal")

# Inheriting from a namedtuple is convenient because it makes the class
# immutable and predefines __init__, __repr__, __hash__, __eq__, and others
class TicTacToeBoard(_TTTB, Node):
    def find_children(board):
        if board.terminal:  # If the game is finished then no moves can be made
            return set()
        # Otherwise, you can make a move in each of the empty spots
        return {
            board.make_move(i) for i, value in enumerate(board.tup) if value is None
        }

    def find_random_child(board):
        if board.terminal:
            return None  # If the game is finished then no moves can be made
        empty_spots = [i for i, value in enumerate(board.tup) if value is None]
        return board.make_move(choice(empty_spots))

    def reward(board):
        if not board.terminal:
            raise RuntimeError(f"reward called on nonterminal board {board}")
        if board.winner is board.turn:
            # It's your turn and you've already won. Should be impossible.
            raise RuntimeError(f"reward called on unreachable board {board}")
        if board.turn is (not board.winner):
            return 0  # Your opponent has just won. Bad.
        if board.winner is None:
            return 0.5  # Board is a tie
        # The winner is neither True, False, nor None
        raise RuntimeError(f"board has unknown winner type {board.winner}")

    def is_terminal(board):
        return board.terminal

    def make_move(board, index):
        tup = board.tup[:index] + (board.turn,) + board.tup[index + 1 :]
        turn = not board.turn
        winner = _find_winner(tup)
        is_terminal = (winner is not None) or not any(v is None for v in tup)
        return TicTacToeBoard(tup, turn, winner, is_terminal)

    def to_pretty_string(board):
        to_char = lambda v: ("X" if v is True else ("O" if v is False else " "))
        rows = [
            [to_char(board.tup[3 * row + col]) for col in range(3)] for row in range(3)
        ]
        return (
            "\n  1 2 3\n"
            + "\n".join(str(i + 1) + " " + " ".join(row) for i, row in enumerate(rows))
            + "\n"
        )


def play_game():
    tree = MCTS()
    board = new_tic_tac_toe_board()
    print(board.to_pretty_string())
    while True:
        row_col = input("enter row,col: ")
        row, col = map(int, row_col.split(","))
        index = 3 * (row - 1) + (col - 1)
        if board.tup[index] is not None:
            raise RuntimeError("Invalid move")
        board = board.make_move(index)
        print(board.to_pretty_string())
        if board.terminal:
            break
        # You can train as you go, or only at the beginning.
        # Here, we train as we go, doing fifty rollouts each turn.
        for _ in range(50):
            tree.do_rollout(board)
        board = tree.choose(board)
        print(board.to_pretty_string())
        if board.terminal:
            break


def _winning_combos():
    for start in range(0, 9, 3):  # three in a row
        yield (start, start + 1, start + 2)
    for start in range(3):  # three in a column
        yield (start, start + 3, start + 6)
    yield (0, 4, 8)  # down-right diagonal
    yield (2, 4, 6)  # down-left diagonal


def _find_winner(tup):
    "Returns None if no winner, True if X wins, False if O wins"
    for i1, i2, i3 in _winning_combos():
        v1, v2, v3 = tup[i1], tup[i2], tup[i3]
        if False is v1 is v2 is v3:
            return False
        if True is v1 is v2 is v3:
            return True
    return None


def new_tic_tac_toe_board():
    return TicTacToeBoard(tup=(None,) * 9, turn=True, winner=None, terminal=False)



# TTB

In [33]:


from collections import namedtuple
from random import choice

_TTTB = namedtuple("TicTacToeBoard", "tup turn winner terminal")

# Inheriting from a namedtuple is convenient because it makes the class
# immutable and predefines __init__, __repr__, __hash__, __eq__, and others
class TicTacToeBoard(_TTTB, Node):
    def find_children(board):
        if board.terminal:  # If the game is finished then no moves can be made
            return set()
        # Otherwise, you can make a move in each of the empty spots
        return {
            board.make_move(i) for i, value in enumerate(board.tup) if value is None
        }

    def find_random_child(board):
        if board.terminal:
            return None  # If the game is finished then no moves can be made
        empty_spots = [i for i, value in enumerate(board.tup) if value is None]
        return board.make_move(choice(empty_spots))

    def reward(board):
        if not board.terminal:
            raise RuntimeError(f"reward called on nonterminal board {board}")
        if board.winner is board.turn:
            # It's your turn and you've already won. Should be impossible.
            raise RuntimeError(f"reward called on unreachable board {board}")
        if board.turn is (not board.winner):
            return 0  # Your opponent has just won. Bad.
        if board.winner is None:
            return 0.5  # Board is a tie
        # The winner is neither True, False, nor None
        raise RuntimeError(f"board has unknown winner type {board.winner}")

    def is_terminal(board):
        return board.terminal

    def make_move(board, index):
        tup = board.tup[:index] + (board.turn,) + board.tup[index + 1 :]
        turn = not board.turn
        winner = _find_winner(tup)
        is_terminal = (winner is not None) or not any(v is None for v in tup)
        return TicTacToeBoard(tup, turn, winner, is_terminal)

    def to_pretty_string(board):
        to_char = lambda v: ("X" if v is True else ("O" if v is False else " "))
        rows = [
            [to_char(board.tup[3 * row + col]) for col in range(3)] for row in range(3)
        ]
        return (
            "\n  1 2 3\n"
            + "\n".join(str(i + 1) + " " + " ".join(row) for i, row in enumerate(rows))
            + "\n"
        )


def play_game():
    tree = MCTS()
    board = new_tic_tac_toe_board()
    print(board.to_pretty_string())
    while True:
        row_col = input("enter row,col: ")
        row, col = map(int, row_col.split(","))
        index = 3 * (row - 1) + (col - 1)
        if board.tup[index] is not None:
            raise RuntimeError("Invalid move")
        board = board.make_move(index)
        print(board.to_pretty_string())
        if board.terminal:
            break
        # You can train as you go, or only at the beginning.
        # Here, we train as we go, doing fifty rollouts each turn.
        for _ in range(50):
            tree.do_rollout(board)
        board = tree.choose(board)
        print(board.to_pretty_string())
        if board.terminal:
            break


def _winning_combos():
    for start in range(0, 9, 3):  # three in a row
        yield (start, start + 1, start + 2)
    for start in range(3):  # three in a column
        yield (start, start + 3, start + 6)
    yield (0, 4, 8)  # down-right diagonal
    yield (2, 4, 6)  # down-left diagonal


def _find_winner(tup):
    "Returns None if no winner, True if X wins, False if O wins"
    for i1, i2, i3 in _winning_combos():
        v1, v2, v3 = tup[i1], tup[i2], tup[i3]
        if False is v1 is v2 is v3:
            return False
        if True is v1 is v2 is v3:
            return True
    return None


def new_tic_tac_toe_board():
    return TicTacToeBoard(tup=(None,) * 9, turn=True, winner=None, terminal=False)




In [98]:
!pip install fnvhash

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fnvhash: filename=fnvhash-0.1.0-py3-none-any.whl size=2157 sha256=725ad795f8f887b904bd9c9ee9214735b7551342f8fcaa7f220ea2b3b3128424
  Stored in directory: c:\users\wubdu\appdata\local\pip\cache\wheels\33\bc\70\488cb8f1353e20bb9c63254b090759282aca093bb47c64aa68
Successfully built fnvhash


In [271]:
class Player():
    def __init__(self, ordinal=0,money=10, hand=None, is_big_blind=False, is_small_blind=False, is_dealer=False):
        self.hand = hand
        self.ordinal = ordinal

        self.is_big_blind = is_big_blind
        self.is_small_blind = is_small_blind
        self.is_dealer= is_dealer
        self.money = money

    # set_role
    def set_role(self, dealer_ordinal, n_players):
        if self.ordinal == dealer_ordinal:
            self.is_dealer = True

        elif (dealer_ordinal + 1) % n_players == self.ordinal:
            self.is_small_blind = True
        elif (dealer_ordinal + 2) % n_players == self.ordinal:
            self.is_big_blind = True



    def __str__(self):
        text = str(self.ordinal) +f"|OP| Hand: {self.hand}, Available Money: {self.money}"
        if self.is_dealer:
            text += ", Dealer"
        if self.is_big_blind:
            text += ", Big Blind"
            
        if self.is_small_blind:
            text += ", Small Blind"
        return text


        
class User():
    def __init__(self,ordinal=0, money=10, hand=None):
        self.hand = hand
        self.ordinal = ordinal
        self.is_big_blind = False
        self.is_small_blind = False
        self.is_dealer= False
        self.money = money

    def call(self):

        return None

    def raise(self, bet):
        #hardcode to 1% 

        return None


    def set_role(self, dealer_ordinal, n_players):
        # dealer
        # Little Blind
        # Big Blind
        if self.ordinal == dealer_ordinal:
            self.is_dealer = True

        elif (dealer_ordinal + 1) % n_players == self.ordinal:
            self.is_small_blind = True
        elif (dealer_ordinal + 2) % n_players == self.ordinal:
            self.is_big_blind = True

    def __str__(self):
        text =  str(self.ordinal) + f"|USER| Hand: {self.hand}, Available Money: {self.money}"
        if self.is_dealer:
            text += ", Dealer"
        if self.is_big_blind:
            text += ", Big Blind"
            
        if self.is_small_blind:
            text += ", Small Blind"
        return text
        
        

IndentationError: expected an indented block after function definition on line 46 (856734820.py, line 50)

In [263]:
player_list = []

# USER INPUTS
my_money = 10
my_hand = ["4s", "2h"]
is_dealer = True

#validate these are real cards
#remove from deck

#just pass in what role you are and it will assume you are not the other roles
player_list = [User(money=my_money, hand=my_hand,is_dealer=is_dealer)]
all_cards = get_deck(my_hand)



#Populate w fake players
for _ in range(5):
    hand = [all_cards.pop(random.randint(0, len(all_cards))) for _ in range(2)]
    player_list.append(Player(money = 10, hand=hand))

TypeError: User.__init__() got an unexpected keyword argument 'is_dealer'

In [264]:

from collections import namedtuple
from random import choice
# from monte_carlo_tree_search import MCTS, Node

_PS = namedtuple("PokerState", "ordered_playerlist pot_size terminal ")

# Inheriting from a namedtuple is convenient because it makes the class
# immutable and predefines __init__, __repr__, __hash__, __eq__, and others
class PokerState(_PS, Node):
    # def __init__(self, ordered_playerlist pot_size terminal ):
    #     # super().__init__()
    #     self.ordered_playerlist=ordered_playerlist
    #     self.pot_size=pot_size
    #     self.terminal=terminal
    
    def find_children():
        
        for player in ordered_playerlist:
            
        #     if not dealer:
        #         player can 
        #         call:
        #         raise:
        #         fold:
                    
        # Rotate positions of dealer, bb and sb
        
        return None


    def find_random_child(board):
        return None


    def reward(board):
        return None


    def is_terminal(board):

        #Round ends when everyone folds except for one player
        return None

    def make_move():

        return None

    def to_pretty_string(self):
        for x in self.ordered_playerlist:
            print(x)
        return None


IndentationError: expected an indented block after 'for' statement on line 18 (3290721955.py, line 28)

In [265]:

def get_deck(my_hand):
    full_deck = []
    for suit in ['s','c','h','d']:
        for num in ["2","3","4","5","6","7","8","9","t","j","q","k","a"]:
            if num+suit not in my_hand:
                full_deck.append(num+suit)
    return full_deck



In [266]:
from fnvhash import fnv1a_32

def hash_value(value):
    hash = hex(fnv1a_32(bytes(value, encoding='utf8')))
    return hash

In [267]:
x = {"breh":"sda"}

for i in x:
    print(str(i))

breh


In [268]:

from collections import namedtuple
from random import choice
# from monte_carlo_tree_search import MCTS, Node


# Inheriting from a namedtuple is convenient because it makes the class
# immutable and predefines __init__, __repr__, __hash__, __eq__, and others

# _PS = namedtuple("PokerState", "ordered_playerlist pot_size terminal ")
_PS = namedtuple("PokerState", "attr_dealer_ordinal attr_playerlist attr_pot_size attr_terminal attr_flop_cards")

class PokerState(_PS,Node):
    def __init__(self, attr_dealer_ordinal, attr_playerlist, attr_pot_size, attr_terminal, attr_flop_cards):
        super().__init__()
        # self.ordered_playerlist=ordered_playerlist
        # self.pot_size=pot_size
        # self.terminal=terminal
        for player in attr_playerlist:
                player.set_role(attr_dealer_ordinal, len(attr_playerlist))

    def __hash__(self):
        attributes = dir(self)
        master_hash_string = ""
        for attr in attributes:
            if attr.startswith("attr_"):
                value = getattr(self, attr)
                if type(value) == list or type(value) == dict:
                    hash_string = ""
                    
                    for x in value:
                        x = str(x)
                        hash_string += hash_value(x)
                else:
                    value = str(value)
                    hash_string = hash_value(value)
                master_hash_string += hash_string
        return master_hash_string

    def __eq__(self,node1, node2):
        if node1.__hash__() != node2.__hash__():
            return False
        return True



    def find_children(self):
        


        #Logic of the progression of the game
        -person after the big blind bets firstr
        

        
        player 1 bets 10
        DEALER
        SB player 2 folds
        BB player 3 bets 11
        # *player can call, raise, fold
        player 1 raises to 12, betting an ADDITIONAL 2  # * round over

        round only stops when everyone has called up to the highest bet
        
        
        for player in attr_playerlist:
            print(player)
        #     if not dealer:
        #         player can 
        #         call:
        #         raise:
        #         fold:
                    
        # Rotate positions of dealer, bb and sb
        
        return None

    def find_random_child(board):
        return None  # If the game is finished then no moves can be made

    def reward(board):
        return None  # If the game is finished then no moves can be made


    def is_terminal(board):
        return None  # If the game is finished then no moves can be made


    # def make_move(board, index):
    #     return None  # If the game is finished then no moves can be made

    

    def __str__(self):
        total_str = ""
        total_str += f"Pot Size: {self.attr_pot_size}\n"
        total_str += f"Flop Cards: {self.attr_flop_cards}\n"
        for x in self.attr_playerlist:
            total_str +=str(x) + "\n"
        return total_str


In [269]:
# Test Equality

new_state1  = PokerState(attr_playerlist=player_list,
 attr_pot_size=1000,
  attr_terminal=False,
   attr_flop_cards=flop_cards)


new_state2  = PokerState(attr_playerlist=player_list,
 attr_pot_size=1000,
  attr_terminal=False,
   attr_flop_cards=flop_cards)

new_state1.__eq__(new_state1, new_state2)

TypeError: PokerState.__new__() missing 1 required positional argument: 'attr_dealer_ordinal'

In [270]:
player_list = []

# USER INPUTS
my_money = 10
my_hand = ["4s", "2h"]
is_dealer = True

#validate these are real cards
#remove from deck

#just pass in what role you are and it will assume you are not the other roles
player_list = [User(ordinal=0,money=my_money, hand=my_hand)]
all_cards = get_deck(my_hand)



#Populate w fake players
for i in range(5):
    hand = [all_cards.pop(random.randint(0, len(all_cards)-1)) for _ in range(2)]
    player_list.append(Player(ordinal=1+i,money = 10, hand=hand))


flop_cards = {}
for _ in range(5):
    flop_cards[all_cards.pop(random.randint(0, len(all_cards)-1))]=False 


    
new_state  = PokerState(attr_playerlist=player_list,
attr_dealer_ordinal=1,
 attr_pot_size=1000,
  attr_terminal=False,
   attr_flop_cards=flop_cards)

print(str(new_state))

Pot Size: 1000
Flop Cards: {'7s': False, '9d': False, 'qd': False, 'qs': False, 'as': False}
0|USER| Hand: ['4s', '2h'], Available Money: 10
1|OP| Hand: ['2s', '8h'], Available Money: 10, Dealer
2|OP| Hand: ['jh', 'kd'], Available Money: 10, Small Blind
3|OP| Hand: ['9h', 'ad'], Available Money: 10, Big Blind
4|OP| Hand: ['8c', 'kc'], Available Money: 10
5|OP| Hand: ['jc', 'ac'], Available Money: 10



In [2]:

from poker_game import generate_game
n_other_players = 5
n_flops = 0


state = generate_game(n_other_players,n_flops )